In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_537110/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2")
        # self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse")
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.4)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_48_4,0.999909,0.827768,0.819369,0.999748,0.000411,1.151718,0.646106,0.000425,0.052947,0.020266,1.000755,0.020619,145.595188,280.635122,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
1,model_1_47_1,0.999906,0.826841,0.817831,0.999726,0.000423,1.157917,0.651609,0.000579,0.059896,0.020560,1.000777,0.020917,145.537727,280.577661,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
2,model_1_47_2,0.999906,0.826885,0.817956,0.999710,0.000425,1.157621,0.651162,0.000614,0.058524,0.020627,1.000783,0.020986,145.524528,280.564462,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
3,model_1_47_4,0.999905,0.826981,0.818138,0.999669,0.000426,1.156978,0.650510,0.000702,0.057341,0.020634,1.000783,0.020993,145.523320,280.563254,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
4,model_1_48_5,0.999905,0.828038,0.819801,0.999583,0.000428,1.149911,0.644562,0.000704,0.052362,0.020694,1.000788,0.021054,145.511697,280.551631,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
491,model_1_0_4,0.918494,0.740640,0.771552,0.938920,0.367151,1.734342,0.817145,0.423278,1.331098,0.605930,1.675335,0.616469,132.003963,267.043897,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
492,model_1_0_3,0.916795,0.738852,0.770847,0.937799,0.374805,1.746297,0.819667,0.431047,1.346947,0.612213,1.689413,0.622861,131.962699,267.002633,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
493,model_1_0_2,0.913936,0.735493,0.770435,0.935223,0.387683,1.768761,0.821142,0.448899,1.343560,0.622642,1.713101,0.633472,131.895133,266.935067,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
494,model_1_0_1,0.899980,0.708601,0.788175,0.916176,0.450548,1.948585,0.757684,0.580893,1.120910,0.671228,1.828733,0.682903,131.594583,266.634517,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
